# Bitcoin Genesis Block Snake Animation Test

This notebook tests the animated genesis block footer that reveals left-to-right in sync with the Platane/snk snake animation.

**Features tested:**
- Animation duration extraction from SVG
- CSS clip-path reveal animation
- Left-alignment with contribution grid
- Duplicate prevention

## 1. Setup - Define the Script

In [ ]:
# add-genesis-footer.py - Full implementation

import re
from pathlib import Path

# Genesis block hex dump - the section containing Satoshi's message
GENESIS_HEX_DUMP = [
    ("00000080", "01 04 45 54 68 65 20 54 69 6D 65 73 20 30 33 2F", "..EThe Times 03/"),
    ("00000090", "4A 61 6E 2F 32 30 30 39 20 43 68 61 6E 63 65 6C", "Jan/2009 Chancel"),
    ("000000A0", "6C 6F 72 20 6F 6E 20 62 72 69 6E 6B 20 6F 66 20", "lor on brink of "),
    ("000000B0", "73 65 63 6F 6E 64 20 62 61 69 6C 6F 75 74 20 66", "second bailout f"),
    ("000000C0", "6F 72 20 62 61 6E 6B 73 FF FF FF FF 01 00 F2 05", "or banksÿÿÿÿ..ò."),
]

# Colors
COLORS = {
    "addr": "#565f89",   # Muted Tokyo Night gray
    "hex": "#a9b1d6",    # Tokyo Night text
    "ascii": "#F7931A",  # Bitcoin orange
}

# Layout constants
FONT_SIZE = 9
LINE_HEIGHT = 11
TOP_PADDING = 10
BOTTOM_PADDING = 5


def parse_viewbox(svg_content: str) -> tuple:
    """Extract viewBox dimensions from SVG (min_x, min_y, width, height)."""
    match = re.search(r'viewBox="([^"]+)"', svg_content)
    if not match:
        raise ValueError("No viewBox found in SVG")
    parts = match.group(1).split()
    return (float(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]))


def find_grid_bounds(svg_content: str) -> tuple:
    """Find the actual contribution grid bounds by looking at rect elements."""
    x_matches = re.findall(r'<rect[^>]*\sx="(\d+)"', svg_content)
    if not x_matches:
        return None
    x_values = [int(x) for x in x_matches]
    min_x = min(x_values)
    max_x = max(x_values) + 12
    return (min_x, max_x)


def extract_animation_duration(svg_content: str) -> int:
    """
    Extract animation duration from snake SVG CSS.
    Platane/snk uses format: animation: none <duration>ms linear infinite
    Returns duration in milliseconds, or 82100 as fallback.
    """
    match = re.search(r'animation:\s*none\s+(\d+)ms', svg_content)
    if match:
        return int(match.group(1))
    return 82100


def update_viewbox(svg_content: str, new_height: float) -> str:
    """Update the viewBox height in the SVG."""
    def replace_viewbox(match):
        parts = match.group(1).split()
        parts[3] = str(int(new_height))
        return f'viewBox="{" ".join(parts)}"'
    return re.sub(r'viewBox="([^"]+)"', replace_viewbox, svg_content)


def update_height_attr(svg_content: str, new_height: float) -> str:
    """Update the height attribute in the SVG root tag only."""
    def replace_in_svg_tag(match):
        svg_tag = match.group(0)
        return re.sub(r'height="[^"]+"', f'height="{int(new_height)}"', svg_tag)
    return re.sub(r'<svg[^>]+>', replace_in_svg_tag, svg_content, count=1)


def generate_hex_dump_svg(grid_left_x: float, start_y: float, duration_ms: int) -> str:
    """Generate SVG elements for the animated hex dump footer."""
    lines = []
    
    # Add style with reveal animation
    style = f'''
  <style type="text/css">
    @keyframes genesis-reveal {{
      0%, 5% {{ clip-path: inset(0 100% 0 0); }}
      95%, 100% {{ clip-path: inset(0 0% 0 0); }}
    }}
    .genesis-block {{
      animation: genesis-reveal {duration_ms}ms linear infinite;
    }}
    .genesis-line {{
      font-family: 'Courier New', Courier, monospace;
      font-size: {FONT_SIZE}px;
      white-space: pre;
    }}
    .genesis-addr {{ fill: {COLORS["addr"]}; }}
    .genesis-hex {{ fill: {COLORS["hex"]}; }}
    .genesis-ascii {{ fill: {COLORS["ascii"]}; }}
  </style>'''
    lines.append(style)
    
    # Create group for hex dump, left-aligned with grid
    lines.append('  <g class="genesis-block">')
    
    char_width = 5.4
    start_x = int(grid_left_x)
    addr_x = start_x
    hex_x = start_x + int(10 * char_width)
    ascii_x = start_x + int(59 * char_width)
    
    for i, (addr, hex_bytes, ascii_text) in enumerate(GENESIS_HEX_DUMP):
        y = int(start_y + (i * LINE_HEIGHT))
        lines.append(f'    <text class="genesis-line genesis-addr" x="{addr_x}" y="{y}">{addr}</text>')
        lines.append(f'    <text class="genesis-line genesis-hex" x="{hex_x}" y="{y}">{hex_bytes}</text>')
        lines.append(f'    <text class="genesis-line genesis-ascii" x="{ascii_x}" y="{y}">{ascii_text}</text>')
    
    lines.append('  </g>')
    return '\n'.join(lines)


def add_genesis_footer(svg_content: str) -> str:
    """Add animated genesis block hex dump footer to SVG content."""
    # Check if genesis block already exists
    if 'class="genesis-block"' in svg_content:
        print("Genesis block already exists, skipping")
        return svg_content
    
    # Parse current viewBox
    min_x, min_y, width, height = parse_viewbox(svg_content)
    
    # Extract animation duration
    duration_ms = extract_animation_duration(svg_content)
    print(f"Extracted animation duration: {duration_ms}ms")
    
    # Calculate new dimensions
    footer_height = TOP_PADDING + (len(GENESIS_HEX_DUMP) * LINE_HEIGHT) + BOTTOM_PADDING
    new_height = height + footer_height
    
    # Find grid bounds for left-alignment
    grid_bounds = find_grid_bounds(svg_content)
    grid_left_x = grid_bounds[0] if grid_bounds else 0
    print(f"Grid left edge: {grid_left_x}")
    
    start_y = min_y + height + TOP_PADDING + FONT_SIZE
    
    # Generate hex dump SVG
    hex_dump_svg = generate_hex_dump_svg(grid_left_x, start_y, duration_ms)
    
    # Update viewBox and height
    svg_content = update_viewbox(svg_content, new_height)
    svg_content = update_height_attr(svg_content, new_height)
    
    # Insert before closing tag
    svg_content = svg_content.replace('</svg>', f'{hex_dump_svg}\n</svg>')
    
    print(f"Added footer. Height: {height} -> {new_height}")
    return svg_content

print("Script loaded successfully!")

## 2. Create a Mock Snake SVG for Testing

In [ ]:
def create_mock_snake_svg(duration_ms: int = 82100) -> str:
    """Create a minimal snake SVG that mimics Platane/snk output."""
    return f'''<svg viewBox="0 0 880 180" width="880" height="180" xmlns="http://www.w3.org/2000/svg">
  <desc>Mock Platane/snk SVG for testing</desc>
  <style>
    :root {{--cb:#1b1f230a;--cs:purple;--ce:#161b22;--c0:#161b22;--c1:#01311f;--c2:#034525;--c3:#0f6d31;--c4:#00c647}}
    .c{{shape-rendering:geometricPrecision;fill:var(--ce);stroke-width:1px;stroke:var(--cb);animation:none {duration_ms}ms linear infinite;width:12px;height:12px}}
    @keyframes c0{{68.81%{{fill:var(--c2)}}68.83%,100%{{fill:var(--ce)}}}}.c.c0{{fill:var(--c2);animation-name:c0}}
    @keyframes c1{{19.97%{{fill:var(--c1)}}19.99%,100%{{fill:var(--ce)}}}}.c.c1{{fill:var(--c1);animation-name:c1}}
  </style>
  <!-- Contribution grid cells -->
  <rect class="c c0" x="16" y="26" width="12" height="12"/>
  <rect class="c c1" x="30" y="26" width="12" height="12"/>
  <rect class="c c0" x="44" y="26" width="12" height="12"/>
  <rect class="c c1" x="58" y="26" width="12" height="12"/>
  <rect class="c c0" x="16" y="40" width="12" height="12"/>
  <rect class="c c1" x="30" y="40" width="12" height="12"/>
  <rect class="c c0" x="832" y="110" width="12" height="12"/>
</svg>'''

# Create and display the mock SVG
mock_svg = create_mock_snake_svg(82100)
print("Mock SVG created!")
print(f"Length: {len(mock_svg)} characters")

## 3. Run Unit Tests

In [ ]:
print("Running unit tests...\n")

# Test 1: Animation duration extraction
svg_75k = create_mock_snake_svg(75000)
duration = extract_animation_duration(svg_75k)
assert duration == 75000, f"Expected 75000, got {duration}"
print("[PASS] Animation duration extraction (75000ms)")

# Test 2: Fallback duration
svg_no_anim = '<svg><rect/></svg>'
duration = extract_animation_duration(svg_no_anim)
assert duration == 82100, f"Expected fallback 82100, got {duration}"
print("[PASS] Fallback duration (82100ms)")

# Test 3: Grid bounds detection
bounds = find_grid_bounds(mock_svg)
assert bounds == (16, 844), f"Expected (16, 844), got {bounds}"
print("[PASS] Grid bounds detection")

# Test 4: ViewBox parsing
viewbox = parse_viewbox(mock_svg)
assert viewbox == (0, 0, 880, 180), f"Expected (0, 0, 880, 180), got {viewbox}"
print("[PASS] ViewBox parsing")

print("\nAll unit tests passed!")

## 4. Generate Animated SVG

In [ ]:
# Apply the genesis footer to our mock SVG
animated_svg = add_genesis_footer(mock_svg)

print("\n--- Validation ---")
print(f"Has @keyframes genesis-reveal: {'@keyframes genesis-reveal' in animated_svg}")
print(f"Has animation timing (82100ms): {'animation: genesis-reveal 82100ms' in animated_svg}")
print(f"Has clip-path start: {'clip-path: inset(0 100% 0 0)' in animated_svg}")
print(f"Has clip-path end: {'clip-path: inset(0 0% 0 0)' in animated_svg}")
print(f"Has genesis-block class: {'class=\"genesis-block\"' in animated_svg}")
print(f"Left-aligned (x=\"16\"): {'x=\"16\"' in animated_svg}")

## 5. Display the Animated SVG

In [ ]:
from IPython.display import SVG, display, HTML

# Display inline - note: animations may not work in all notebook viewers
display(HTML(f'<div style="background: #161b22; padding: 20px; border-radius: 8px;">{animated_svg}</div>'))

## 6. Test Duplicate Prevention

In [ ]:
# Try adding footer again - should be skipped
print("Attempting to add footer to SVG that already has one...")
result = add_genesis_footer(animated_svg)

# Count genesis-block occurrences
count = result.count('class="genesis-block"')
print(f"\nGenesis block count: {count}")
assert count == 1, f"Expected 1 genesis-block, got {count}"
print("[PASS] Duplicate prevention works!")

## 7. Download the Generated SVG

In [ ]:
# Save to file for download
with open('genesis-animation-test.svg', 'w') as f:
    f.write(animated_svg)

print("SVG saved to 'genesis-animation-test.svg'")
print("\nTo view the animation:")
print("1. Download the file")
print("2. Open in a web browser")
print("3. Watch the hex dump reveal left-to-right!")

# For Google Colab - create download link
try:
    from google.colab import files
    files.download('genesis-animation-test.svg')
except ImportError:
    print("\n(Not running in Colab - file saved locally)")

## 8. View Raw Generated CSS

In [ ]:
# Extract and display just the genesis CSS
import re

genesis_style_match = re.search(r'<style type="text/css">\s*(@keyframes genesis-reveal.*?)</style>', 
                                 animated_svg, re.DOTALL)
if genesis_style_match:
    print("Generated Animation CSS:")
    print("=" * 50)
    print(genesis_style_match.group(1))
else:
    print("Could not extract CSS")

## 9. Animation Timeline Visualization

In [ ]:
# Visualize the animation timeline
duration_ms = 82100

print("Animation Timeline (82.1 seconds total)")
print("=" * 60)
print("")
print("0%   5%                                              95%  100%")
print("|----+--------------------------------------------------+----|") 
print("     |                                                  |")
print("     |<-- Genesis text reveals left-to-right --------->|")
print("     |                                                  |")
print(f"     {int(duration_ms * 0.05)}ms                                          {int(duration_ms * 0.95)}ms")
print("")
print("Timeline breakdown:")
print(f"  0-5%:    Hidden (clip-path: inset(0 100% 0 0)) - {int(duration_ms * 0.05)}ms")
print(f"  5-95%:   Revealing animation - {int(duration_ms * 0.90)}ms")
print(f"  95-100%: Fully visible (clip-path: inset(0 0% 0 0)) - {int(duration_ms * 0.05)}ms")
print(f"  Loop:    Animation resets and repeats infinitely")

---

## Summary

The animated genesis block footer:

1. **Extracts** the snake animation duration from the SVG CSS (82100ms default)
2. **Reveals** the hex dump text left-to-right using CSS `clip-path` animation
3. **Syncs** with the snake animation (starts at 5%, completes at 95%)
4. **Left-aligns** with the contribution grid's left edge
5. **Loops** cleanly with the snake animation reset
6. **Prevents duplicates** if run multiple times